# Meat Quality Assessment - Using our model to make new predictions

---

In the last step of our project we use our trained model from the previous step to make predictions on new images.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from functools import partial

import numpy as np
import pandas as pd
import tensorflow as tf

from loguru import logger

# Set parent folder as root to import local modules
module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)

# Remove default logger and set level to INFO
logger.remove()
logger.add(sys.stderr, level="INFO")

from src.base.pipelines import decode_image

In [ ]:
data_path = os.path.join(module_path, "data/meat-quality-assessment-based-on-deep-learning/")
output_path = os.path.join(module_path, "notebooks/output/")

In [ ]:
model = tf.keras.models.load_model(output_path)

In [ ]:
test_df = pd.read_csv(os.path.join(output_path, "test_df.csv"), index_col=None)
test_df.head()

In [ ]:
BATCH_SIZE = 32
RESIZE_SHAPE = (256, 256)
SHUFFLE_SIZE = 512

In [ ]:
test_set = (
    tf.data.Dataset
    .from_tensor_slices((test_df["image_path"], test_df["target"]))
    .map(partial(decode_image, image_size=RESIZE_SHAPE, n_labels=2), num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
preds_proba = model.predict(test_set)

In [ ]:
print(test_df.shape, preds_proba.shape)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_true=test_df["target"].values, y_pred=np.argmax(preds_proba, axis=1))